In [21]:
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# import nltk

In [ ]:
!pip install datasets

In [ ]:
!pip install transformers

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install accelerate

  Using cached accelerate-0.29.2-py3-none-any.whl (297 kB)


In [ ]:
!pip install rouge_score

In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00


In [22]:
# import transformers
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments, Seq2SeqTrainingArguments
from datasets import load_dataset, load_from_disk, Dataset
from sklearn.model_selection import train_test_split

In [23]:
import accelerate

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
# your PyTorch code

In [ ]:
# import transformers
# import accelerate
# import torch

# print("Transformers version:", transformers.__version__)
# print("Accelerate version:", accelerate.__version__)
# print("PyTorch version:", torch.__version__)

Transformers version: 4.38.2
Accelerate version: 0.29.2
PyTorch version: 2.2.1+cu121


In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
data = pd.read_csv("/content/drive/My Drive/training_QAdata.csv")

In [ ]:
data.head()
# print(data['answer'][0])

,question,answer
0,"What dishes can we make with kumquats , gallon...",You can make Pork Porterhouse With Kumquat Chu...
1,"What dishes can we make with bourbon , maple s...",You can make The Eldred . Here is the recipe ...
2,"What dishes can we make with vanilla , heavy c...",You can make Low Carb Neapolitan Parfaits . H...
3,"What dishes can we make with cumin , oregano ,...",You can make Arizona Albondigas Soup Recipe . ...
4,What dishes can we make with additional whippe...,You can make Mint Chocolate Pudding Shots . H...


In [26]:
df = pd.DataFrame(data)
df_random_8000 = df.sample(n=8000, random_state=42)
df = df_random_8000
train_df, test_df = train_test_split(df, test_size=0.2)
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [27]:
# max_input = 512
# max_target = 128
# batch_size = 3
max_length = 512
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

In [28]:
def preprocess_function(examples):
    inputs = [q for q in examples['question']]
    model_inputs = tokenizer(inputs,padding='max_length', max_length=max_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['answer'], padding='max_length', max_length=max_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_tokenized = train_dataset.map(preprocess_function, batched=True)
test_tokenized = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/6400 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

In [29]:
train_tokenized.save_to_disk('/content/drive/My Drive/train_tokenized')
test_tokenized.save_to_disk ('/content/drive/My Drive/test_tokenized')

Saving the dataset (0/1 shards):   0%|          | 0/6400 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1600 [00:00<?, ? examples/s]

In [30]:
from datasets import load_from_disk

# Load the dataset from the disk
train_tokenized = load_from_disk('/content/drive/My Drive/train_tokenized')
test_tokenized = load_from_disk('/content/drive/My Drive/test_tokenized')

In [31]:
# from transformers import Trainer, TrainingArguments
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer


training_args = Seq2SeqTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=2,   # batch size for training
    per_device_eval_batch_size=2,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    gradient_accumulation_steps=8,
    learning_rate=3e-4,
)

In [32]:
import nltk, evaluate
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # rougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return result


In [33]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    tokenizer=tokenizer,
    compute_metrics=metric
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [34]:
trainer.train()

Step,Training Loss
10,16.399100
20,12.266800
30,10.375900
40,8.910900
50,7.747200
60,6.027800
70,4.996000
80,4.255700
90,3.341400
100,2.294700


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


TrainOutput(global_step=1200, training_loss=1.183693216641744, metrics={'train_runtime': 8322.6539, 'train_samples_per_second': 2.307, 'train_steps_per_second': 0.144, 'total_flos': 2.08042041802752e+16, 'train_loss': 1.183693216641744, 'epoch': 3.0})

In [36]:
trainer.evaluate()

OutOfMemoryError: CUDA out of memory. Tried to allocate 4.79 GiB. GPU 0 has a total capacity of 14.75 GiB of which 4.74 GiB is free. Process 224543 has 10.00 GiB memory in use. Of the allocated memory 9.45 GiB is allocated by PyTorch, and 435.02 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [42]:
model.save_pretrained('./saved_model/bart8000')
tokenizer.save_pretrained('./saved_model/bart8000')


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('./saved_model/bart8000/tokenizer_config.json',
 './saved_model/bart8000/special_tokens_map.json',
 './saved_model/bart8000/vocab.json',
 './saved_model/bart8000/merges.txt',
 './saved_model/bart8000/added_tokens.json')

In [46]:
# from transformers import BartForConditionalGeneration, BartTokenizer

model = BartForConditionalGeneration.from_pretrained('./saved_model/bart8000')
tokenizer = BartTokenizer.from_pretrained('./saved_model/bart8000')

# Encode input context
input_text = "what dishes can we make with beef and tomato"
inputs = tokenizer(input_text, return_tensors="pt")

# # Generate a response
# outputs = model.generate(inputs['input_ids'], num_beams=5, early_stopping=True)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# Debug: Check input tokens
# print("Tokenized inputs:", tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))

outputs = model.generate(**inputs, max_length=512)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Debug: Check generated text
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))


['You can make Baked Potatoes.  Here is the recipe : Preheat oven to 350 degrees F (175 degrees C).  Line a baking sheet with parchment paper.  Place potatoes in a single layer on the prepared baking sheet.  Bake in the preheated oven for 20 minutes, or until tender.  Remove from oven and let cool slightly.  In a small bowl, whisk together the flour, baking powder, salt, and pepper.  Add potatoes to the flour mixture and toss to coat.  Transfer to a large bowl and set aside.  Heat oil in a large skillet over medium-high heat.  Cook potatoes in batches, stirring occasionally, until golden brown and crisp-tender, about 5 minutes.  Drain on paper towels. ']
